In [ ]:
#### first example i found... this first one is pretty cool i think....you kind of slide it 
### around 

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons



fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.25)
t = np.arange(0.0, 1.0, 0.001)
a0 = 5
f0 = 3
s = a0*np.sin(2*np.pi*f0*t)
l, = plt.plot(t, s, lw=1, color='red')
plt.axis([0, 1, -10, 10])

axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], axisbg=axcolor)
axamp = plt.axes([0.25, 0.15, 0.65, 0.03], axisbg=axcolor)

sfreq = Slider(axfreq, 'Freq', 0.1, 30.0, valinit=f0)
samp = Slider(axamp, 'Amp', 0.1, 10.0, valinit=a0)


def update(val):
    amp = samp.val
    freq = sfreq.val
    l.set_ydata(amp*np.sin(2*np.pi*freq*t))
    fig.canvas.draw_idle()
sfreq.on_changed(update)
samp.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    sfreq.reset()
    samp.reset()
button.on_clicked(reset)

rax = plt.axes([0.025, 0.5, 0.15, 0.15], axisbg=axcolor)
radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)


def colorfunc(label):
    l.set_color(label)
    fig.canvas.draw_idle()
radio.on_clicked(colorfunc)

plt.show()



In [ ]:
#second example i found....recording 'x' and 'y' 
#....but they dont really correspond to axis like in other examples


from Tkinter import * 

def onObjectClick(event):                  
    print('Got object click', event.x, event.y)
    print(event.widget.find_closest(event.x, event.y))

root = Tk()
canv = Canvas(root, width=100, height=100)
obj1Id = canv.create_line(0, 30, 100, 30, width=5, tags="obj1Tag")
obj2Id = canv.create_text(50, 70, text='Click', tags='obj2Tag')

canv.tag_bind(obj1Id, '<ButtonPress-1>', onObjectClick)       
canv.tag_bind('obj2Tag', '<ButtonPress-1>', onObjectClick)   
print('obj1Id: ', obj1Id)
print('obj2Id: ', obj2Id)
canv.pack()
root.mainloop()

In [ ]:
# this one tracks the x,y coordinates properly, but has no 'event' for the click

########################  i'm working on this version
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

from numpy import arange, sin, pi
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
# implement the default mpl key bindings
from matplotlib.backend_bases import key_press_handler


from matplotlib.figure import Figure

import sys
if sys.version_info[0] < 3:
    import Tkinter as Tk
else:
    import tkinter as Tk

root = Tk.Tk()
root.wm_title("Embedding in TK")


f = Figure(figsize=(5, 4), dpi=100)
#a = f.add_subplot(111)
#t = arange(0.0, 3.0, 0.01)
#s = sin(2*pi*t)

#a.plot(t, s)

delta = 0.025
x = y = np.arange(-3.0, 3.01, delta)
X, Y = np.meshgrid(x, y)
Z = X**2+Y**2
CS = plt.contourf(X, Y, Z)
plt.colorbar()
plt.show()


# a tk.DrawingArea
canvas = FigureCanvasTkAgg(f, master=root)
canvas.show()
canvas.get_tk_widget().pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)

toolbar = NavigationToolbar2TkAgg(canvas, root)
toolbar.update()
canvas._tkcanvas.pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)


#def onclick(event):
#    print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
#          (event.button, event.x, event.y, event.xdata, event.ydata))

#cid = f.canvas.mpl_connect('button_press_event', onclick)


def on_key_event(event):
   
    print('you pressed %s' % event.key)
    key_press_handler(event, canvas, toolbar)

canvas.mpl_connect('key_press_event', on_key_event)


def _quit():
    root.quit()     # stops mainloop
    root.destroy()  # this is necessary on Windows to prevent
                    # Fatal Python Error: PyEval_RestoreThread: NULL tstate

button = Tk.Button(master=root, text='Quit', command=_quit)
button.pack(side=Tk.BOTTOM)

Tk.mainloop()
# If you put root.destroy() here, it will cause an error if
# the window is closed with the window manager.



In [ ]:
# i don't think this one is useful

#!/usr/bin/env python
# -*- noplot -*-

import matplotlib as mpl
import numpy as np
import sys
if sys.version_info[0] < 3:
    import Tkinter as tk
else:
    import tkinter as tk
import matplotlib.backends.tkagg as tkagg
from matplotlib.backends.backend_agg import FigureCanvasAgg


def draw_figure(canvas, figure, loc=(0, 0)):
    """ Draw a matplotlib figure onto a Tk canvas

    loc: location of top-left corner of figure on canvas in pixels.

    Inspired by matplotlib source: lib/matplotlib/backends/backend_tkagg.py
    """
    figure_canvas_agg = FigureCanvasAgg(figure)
    figure_canvas_agg.draw()
    figure_x, figure_y, figure_w, figure_h = figure.bbox.bounds
    figure_w, figure_h = int(figure_w), int(figure_h)
    photo = tk.PhotoImage(master=canvas, width=figure_w, height=figure_h)

    # Position: convert from top-left anchor to center anchor
    canvas.create_image(loc[0] + figure_w/2, loc[1] + figure_h/2, image=photo)

    # Unfortunatly, there's no accessor for the pointer to the native renderer
    tkagg.blit(photo, figure_canvas_agg.get_renderer()._renderer, colormode=2)

    # Return a handle which contains a reference to the photo object
    # which must be kept live or else the picture disappears
    return photo

# Create a canvas
w, h = 300, 200
window = tk.Tk()
window.title("A figure in a canvas")
canvas = tk.Canvas(window, width=w, height=h)
canvas.pack()

# Generate some example data
X = np.linspace(0, 2.0*3.14, 50)
Y = np.sin(X)

# Create the figure we desire to add to an existing canvas
fig = mpl.figure.Figure(figsize=(2, 1))
ax = fig.add_axes([0, 0, 1, 1])
ax.plot(X, Y)

# Keep this handle alive, or else figure will disappear
fig_x, fig_y = 100, 100
fig_photo = draw_figure(canvas, fig, loc=(fig_x, fig_y))
fig_w, fig_h = fig_photo.width(), fig_photo.height()

# Add more elements to the canvas, potentially on top of the figure
canvas.create_line(200, 50, fig_x + fig_w / 2, fig_y + fig_h / 2)
canvas.create_text(200, 50, text="Zero-crossing", anchor="s")

# Let Tk take over
tk.mainloop()

In [ ]:
# this one tracks x,y properly but with no click events


#!/usr/bin/env python

import matplotlib
matplotlib.use('TkAgg')

from numpy import arange, sin, pi
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
# implement the default mpl key bindings
from matplotlib.backend_bases import key_press_handler


from matplotlib.figure import Figure

import sys
if sys.version_info[0] < 3:
    import Tkinter as Tk
else:
    import tkinter as Tk

root = Tk.Tk()
root.wm_title("Embedding in TK")


f = Figure(figsize=(5, 4), dpi=100)
a = f.add_subplot(111)
t = arange(0.0, 3.0, 0.01)
s = sin(2*pi*t)

a.plot(t, s)


# a tk.DrawingArea
canvas = FigureCanvasTkAgg(f, master=root)
canvas.show()
canvas.get_tk_widget().pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)

toolbar = NavigationToolbar2TkAgg(canvas, root)
toolbar.update()
canvas._tkcanvas.pack(side=Tk.TOP, fill=Tk.BOTH, expand=1)


def on_key_event(event):
    print('you pressed %s' % event.key)
    key_press_handler(event, canvas, toolbar)

canvas.mpl_connect('key_press_event', on_key_event)


def _quit():
    root.quit()     # stops mainloop
    root.destroy()  # this is necessary on Windows to prevent
                    # Fatal Python Error: PyEval_RestoreThread: NULL tstate

button = Tk.Button(master=root, text='Quit', command=_quit)
button.pack(side=Tk.BOTTOM)

Tk.mainloop()
# If you put root.destroy() here, it will cause an error if
# the window is closed with the window manager.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.random.rand(10))

def onclick(event):
    print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))

cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()

In [ ]:
from matplotlib import pyplot as plt

class LineBuilder:
    def __init__(self, line):
        self.line = line
        self.xs = list(line.get_xdata())
        self.ys = list(line.get_ydata())
        self.cid = line.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        print('click', event)
        if event.inaxes!=self.line.axes: return
        self.xs.append(event.xdata)
        self.ys.append(event.ydata)
        self.line.set_data(self.xs, self.ys)
        self.line.figure.canvas.draw()

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('click to build line segments')
line, = ax.plot([0], [0])  # empty line
linebuilder = LineBuilder(line)

plt.show()

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.random.rand(10))

def onclick(event):
    print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))

cid = fig.canvas.mpl_connect('button_press_event', onclick)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


x = np.arange(-10,10)
y = x**2

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x,y)

coords = []

def onclick(event):
    global ix, iy
    ix, iy = event.xdata, event.ydata
    print 'x = %d, y = %d'%(
        ix, iy)

    global coords
    coords.append((ix, iy))

    if len(coords) == 2:
        fig.canvas.mpl_disconnect(cid)

    return coords
cid = fig.canvas.mpl_connect('button_press_event', onclick)
        


In [ ]:
##### matplotlib has their own event handler!!!! maybe it will be sufficient

import matplotlib.pyplot as plt

fig = plt.figure()
#ax = fig.add_subplot(111)
#ax.plot(np.random.rand(10))

type(fig)

delta = 0.025
x = y = np.arange(-3.0, 3.01, delta)
X, Y = np.meshgrid(x, y)
Z = X**2+Y**2
CS = plt.contourf(X, Y, Z)
plt.colorbar()
#plt.show()

#print type(CS)

def onclick(event):
    print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))

cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()

In [ ]:

####### working version here, messy one below



# this changes in real time, accoridn to how slider is moved. so all i have to do is link this
# ...up with event clicker of other map. if i click on say (3,5) on heat map then i need the
# ...sliders to effectivley move to those numbers

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons




fig, ax = plt.subplots()                       #uses subplots function

# fig is a 'figure object' (like blank canvas?), 
#....ax is a list of axis objects (things you put on canvas?)

# figure is the whole canvas and can contain multiple graphs....
#... for example 

#fig, ax = plt.subplots()

#plt.subplots_adjust(left=0.25, bottom=0.25)



#controls lenght of line segment that you are graphing as 'line'
t = np.arange(-30.0, 30.0, 0.001)
a0 = 5
f0 = 3
#s = a0*np.sin(2*np.pi*f0*t)
s = a0 + f0*t

#t = np.concatenate( (t, np.array([-5,0,5]) ))
#s = np.concatenate((s, np.array([5,10,0])))


#ax = fig.add_subplot(111)

#  add_subplot

#data_x = [-5, 0, 5]
#data_y = [5, 10,0]
#for xy in zip(A, B):                                       # <--
#    ax.annotate('(%s, %s)' % xy, xy=xy, textcoords='data') # <--


#ax.plot(data_x, data_y, 'or')

l, = plt.plot(t, s, lw=2, color='black')
plt.axis([-10, 10, -10, 10])











#axcolor = 'lightgoldenrodyellow'
#axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], axisbg=axcolor)
#axamp = plt.axes([0.25, 0.15, 0.65, 0.03], axisbg=axcolor)




#sfreq = Slider(axfreq, 'Freq', -10, 10.0, valinit=f0)
#samp = Slider(axamp, 'Amp', -10, 10.0, valinit=a0)




#def update(val):
#    amp = samp.val
#    freq = sfreq.val
#    # i think probably: 
#    #amp = event.xdata 
#    #freq = event.ydata  #using notation from other script
#   
#    #l.set_ydata(amp*np.sin(2*np.pi*freq*t))
#    l.set_ydata(amp + freq*t)
#    fig.canvas.draw_idle()
    

#sfreq.on_changed(update)
#samp.on_changed(update)
    




    
    ####################################################################
####################################################################
figg = plt.figure()
#ax = figg.add_subplot(111)
#ax.plot(np.random.rand(10))


delta = 0.025
x = y = np.arange(-3.0, 3.01, delta)
X, Y = np.meshgrid(x, y)
Z = X**2+Y**2
CS = plt.contourf(X, Y, Z)
plt.colorbar()
#plt.show()


def onclick(event):
    #print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
    #      (event.button, event.x, event.y, event.xdata, event.ydata))
    if (event.xdata != None) and (event.ydata != None):
        #print event.xdata
        l.set_ydata(event.ydata + event.xdata*t)
        fig.canvas.draw_idle()
    
    
    #sfreq.val = event.xdata
    #samp.val = event.ydata
    #sfreq.on_changed(update)
    #samp.on_changed(update)
    #print sfreq.val
    
cid = figg.canvas.mpl_connect('button_press_event', onclick)
###################################################################
###################################################################


    
    
    
    


#### just deals with reset button
#resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
#button = Button(resetax, 'Reset', color='black', hovercolor='0.975')

#def reset(event):
#    sfreq.reset()
#    samp.reset()
#button.on_clicked(reset)
###################################


#### probably just for little color legend
#rax = plt.axes([0.025, 0.5, 0.15, 0.15], axisbg=axcolor)
#radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)

#def colorfunc(label):
#    l.set_color(label)
#    fig.canvas.draw_idle()
#radio.on_clicked(colorfunc)
######################################

plt.show()



############  to do
#  put plots on single page .subplot
#  add dot with coordinates when i click heat map
#   add data points with vertical lines (labeled with lengths) 
#
#

In [ ]:
from matplotlib import pyplot as plt

#fig = plt.figure()

ax = fig.add_subplot(111)  ## adding feature to the 'canvas' fig

A = -0.75, -0.25, 0, 0.25, 0.5, 0.75, 1.0
B = 0.73, 0.97, 1.0, 0.97, 0.88, 0.73, 0.54

plt.plot(A,B)

for xy in zip(A, B):                                       # <--
    ax.annotate('(%s, %s)' % xy, xy=xy, textcoords='data') # <--

plt.grid()
plt.show()

In [5]:


####### test version



# this changes in real time, accoridn to how slider is moved. so all i have to do is link this
# ...up with event clicker of other map. if i click on say (3,5) on heat map then i need the
# ...sliders to effectivley move to those numbers

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons


fig = plt.figure()

#fig, ax = plt.subplots()                       #uses subplots function

# fig is a 'figure object' (like blank canvas?), 
#....ax is a list of axis objects (things you put on canvas?)

# figure is the whole canvas and can contain multiple graphs....
#... for example 

#fig, ax = plt.subplots()

#plt.subplots_adjust(left=0.25, bottom=0.25)



#controls lenght of line segment that you are graphing as 'line'
t = np.arange(-30.0, 30.0, 0.001)
a0 = 0
f0 = 0
#s = a0*np.sin(2*np.pi*f0*t)
s = a0 + f0*t

#t = np.concatenate( (t, np.array([-5,0,5]) ))
#s = np.concatenate((s, np.array([5,10,0])))


ax = fig.add_subplot(223)

#  add_subplot

#ax.plot([])

data_x = [-5, 0, 5]
data_y = [5, 8,0]
for xy in zip(data_x, data_y):         # <--
    ax.annotate('(%s, %s)' % xy, xy=xy, textcoords='data') # <--
    ax.scatter(data_x, data_y)

#ax.plot(data_x, data_y, 'or')
#ax.set_title('  agffsdgaf      ')

ax.set_xlabel('x')
ax.set_ylabel('y')

p1 = 0
p2 = 0
p3= 0

segment1, = plt.plot([-5, -5], [p1, 5], 'r-', lw=1)


segment2, = plt.plot([0, 0], [p2, 8], 'r-', lw=1)


segment3, = plt.plot([5, 5], [p3, 0], 'r-', lw=1)



# why do we need that comma after segment?

# note its ordered like (x1, x2)  (y1, y2) where we want (x1,y1),(x2,y2)



l, = plt.plot(t, s, lw=2, color='black')
plt.axis([-10, 10, -10, 10])


#l.xdata



#axcolor = 'lightgoldenrodyellow'
#axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], axisbg=axcolor)
#axamp = plt.axes([0.25, 0.15, 0.65, 0.03], axisbg=axcolor)




#sfreq = Slider(axfreq, 'Freq', -10, 10.0, valinit=f0)
#samp = Slider(axamp, 'Amp', -10, 10.0, valinit=a0)




#def update(val):
#    amp = samp.val
#    freq = sfreq.val
#    # i think probably: 
#    #amp = event.xdata 
#    #freq = event.ydata  #using notation from other script
#   
#    #l.set_ydata(amp*np.sin(2*np.pi*freq*t))
#    l.set_ydata(amp + freq*t)
#    fig.canvas.draw_idle()
    

#sfreq.on_changed(update)
#samp.on_changed(update)
    




    
    ####################################################################
####################################################################
#figg = plt.figure()
#axx = figg.add_subplot(221)

axx = fig.add_subplot(221)

#ax.plot(np.random.rand(10))

# 122 means: break the canvas up into a 1x2 grid and draw this stuff..
#...in the 2nd square

delta = 0.025
x = y = np.arange(-3.0, 3.01, delta)
X, Y = np.meshgrid(x, y)
Z = X**2+Y**2
CS = plt.contourf(X, Y, Z)
plt.colorbar()
#plt.show()

#axx.set_title('axes title')

axx.set_xlabel('m')
axx.set_ylabel('b')
#    global flag 
#  flag = 0

#axx.invert_yaxis()
#axx.xaxis.tick_top()


coord = axx.annotate('' , xy=(0,0), textcoords='data') # <--
coord1 = axx.annotate('' , xy=(0,0), textcoords='data') # <--


#hello = 5

def onclick(event):
    #print('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
    #      (event.button, event.x, event.y, event.xdata, event.ydata))
    
    

    if (event.xdata != None) and (event.ydata != None):
        
        #print event.xdata
        l.set_ydata(event.ydata + event.xdata*t)
        
        segment1.set_xdata( [-5,-5])
        
        segment1.set_ydata([event.ydata + event.xdata*(-5),5])
        
        #ax.annotate('local max', xy=(2, 1), xytext=(3, 1.5),arrowprops=dict(facecolor='black', shrink=0.05),)
        
        
        ##################### the 4 below will tell you error of first segment
        #global coord1
        #coord1.remove()
        #seg_len_1 = round(np.absolute( event.ydata + event.xdata*(-5) - 5) ,2 )
        #coord1 = ax.annotate('%.2f'%seg_len_1 , xy=(-5, 5-seg_len_1),xytext=(-7, 5 - seg_len_1 ),color='red', fontsize=8)
        
        
        segment2.set_xdata( [0,0])
        
        segment2.set_ydata([event.ydata + event.xdata*(0),8])
        
        segment3.set_xdata( [5,5])
        
        segment3.set_ydata([event.ydata + event.xdata*(5),0])
        
        #plt.plot([p1, -5], [p2, 5], 'r-', lw=2)

        fig.canvas.draw_idle()  # 'fig' is name of specific object
        #####################################################
        
        
        error = event.xdata   ## just testing to see if text chagnes
        
        txt = 'The error (SSE) is \n \n (-5m+b-5)^2+(0m+b-8)^2+(5m+b-0)^2 \n \n =%f'%error
        
        txt_message.set_text(txt)
        
        #txt = 'The error (SSE) is \n (-5m+b-5)^2+(0m+b-8)^2+(5m+b-0)^2=%d'%error
        #txt_message = fig.text(.5,.4,txt)
        
        ############################################
        xy = (event.xdata, event.ydata)
        
        
        #if flag == 1:
        #    new.remove()
        #lag = 1
        
        
        #data_x = [event.xdata]
        #data_y = [event.ydata]
        #for xy in zip(data_x, data_y):         # <--
        
        
        #if 'coord' not in locals():
        #    print 'not in local'
        #    #coord.remove()
        
        #if 'coord' in locals():
        #    print 'in local'
        #    coord.remove()
        #    del coord
       
        global coord
        coord.remove()
        coord = axx.annotate('(%s, %s)' % (round(xy[0],2), round(xy[1],2)), xy=xy, textcoords='data') # <--
###holy shit, the above worked...I guess when i click, my previous
#experience suggests that python reads the contents of the function 
#'onclick' before anything else, so it views coord as a local variable
#defined within the function.  by tellign python its a global variable,
#when it reaches 'coord.remove' it looks for the meaning of coord not 
#just locally, but with 'maximal scope'. actually: it must FIRST
#    look locally, and when that fails, it looks globally.  thats the
#    only thing that can explaine some of this behavior. so now when 
#    it first reaches coord and can't' find the def locally, it uses
#    the global version... but after coord is defined locally the first 
#   time, python only ever refers to that local definition from that 
#    point on
        
        #figg.canvas.draw_idle()  # 'fig' is name of specific object
        fig.canvas.draw_idle()
    
    
    #sfreq.val = event.xdata
    #samp.val = event.ydata
    #sfreq.on_changed(update)
    #samp.on_changed(update)
    #print sfreq.val
cid = fig.canvas.mpl_connect('button_press_event', onclick)




error = 1000

txt = 'The error (SSE) is \n \n (-5m+b-5)^2+(0m+b-8)^2+(5m+b-0)^2 \n \n =%f'%error
txt_message = fig.text(.6,.7,txt)



txt1 = 'y=mx+b \n \n \n m=%f \n b=%f'%(round(error,2), round(error,2))
txt_message1 = fig.text(.6,.3,txt1,fontsize=12)



###################################################################
###################################################################


    
    
    
    


#### just deals with reset button
#resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
#button = Button(resetax, 'Reset', color='black', hovercolor='0.975')

#def reset(event):
#    sfreq.reset()
#    samp.reset()
#button.on_clicked(reset)
###################################


#### probably just for little color legend
#rax = plt.axes([0.025, 0.5, 0.15, 0.15], axisbg=axcolor)
#radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)

#def colorfunc(label):
#    l.set_color(label)
#    fig.canvas.draw_idle()
#radio.on_clicked(colorfunc)
######################################
#fig.tight_layout() 
plt.show()



############  to do
#  put plots on single page .subplot
#  add dot with coordinates when i click heat map
#   add data points with vertical lines (labeled with lengths) 
#
#


#Your goal is to find two numbers m and b that make the error
#as small as possible. The error is determined by two numbers, m and b (as you 
#cansee from the formula).
#Every time you click on the mb-plane, you are making a choice
#of m and b that determine a linear model y=mx+b of your data. The error
#of this model is plotted  below

# the contour map helps you decide where to click to obtain a small
#error and hence helps you decide 
#which values of m and b give you the best model.

## make sure i'm using correct cost function
